In [1]:
import pytesseract
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import requests
import os
from pdf2image import convert_from_path, convert_from_bytes

from pdf2image.exceptions import (
    PDFInfoNotInstalledError,
    PDFPageCountError,
    PDFSyntaxError
)

from main import transfer26, pdfDocToString, shortening, text2spans26

# first level of shortening
import spacy as spacy
import json
from spacy.tokens import Span
from spacy.matcher import Matcher

In [2]:
fName = r"F:\Python\OCR_diploma\docs_base\StauffInvoices\92951827.pdf" 
fName = r"F:\Python\OCR_diploma\docs_base\StauffInvoices\92951828.pdf"
fName = r"F:\Python\OCR_diploma\docs_base\StauffInvoices\92946889.pdf"


In [3]:
fName = r"F:\Python\OCR_diploma\docs_base\StauffInvoices\92951828.pdf"
fName = r"F:\Python\OCR_diploma\docs_base\StauffInvoices\92946891.pdf"
transfer26(fName=fName)

'[[["NORDER", "2360210"], ["CONTRACT", "SR-1-06"], ["CONTRACT1", "1577"], ["POS", "11200"], ["AMOUNT", "120"], ["ARTICLE", "6010001432"], ["PRICE", "187,55"], ["UNIT", "100"], ["SUMMA", "225,06"], ["TARIFF", "73079910"], ["COUNTRY", "Germany\\n\\n"], ["PR_SURCH", "6,90"], ["SURCHARGE", "15,53"], ["TOTAL", "240,59"]], [["NORDER", "2360211"], ["CONTRACT", "SR-1-06"], ["CONTRACT1", "1578"], ["POS", "11300"], ["AMOUNT", "6.600"], ["ARTICLE", "1120021036"], ["PRICE", "23,81"], ["UNIT", "100"], ["SUMMA", "1.571,46"], ["TARIFF", "73269098"], ["COUNTRY", "Germany\\n\\n"], ["PR_SURCH", "11,40"], ["SURCHARGE", "179,15"], ["TOTAL", "1.750,61"]], [["NORDER", "2367112"], ["CONTRACT", "SR-1-06"], ["CONTRACT1", "1662"], ["POS", "11400"], ["AMOUNT", "60"], ["ARTICLE", "6020000526"], ["PRICE", "135,77"], ["UNIT", "100"], ["SUMMA", "81,46"], ["TARIFF", "73079910"], ["COUNTRY", "Germany\\n\\n"], ["PR_SURCH", "6,90"], ["SURCHARGE", "5,62"], ["TOTAL", "87,08"]], [["NORDER", "2367113"], ["CONTRACT", "SR-1-0

In [13]:
text = pdfDocToString(f_name=fName)
txtS = shortening(text)
docs = text2spans26(txtS)

In [15]:
fName

'F:\\Python\\OCR_diploma\\docs_base\\StauffInvoices\\92951828.pdf'

In [14]:
#text = pdfDocToString(f_name=fName)
#txtS = shortening(text)
#docs = text2spans26(txtS)
nlp_ = spacy.load(r"C:\Users\AdminAsus\source\python\spacyST\spacySt-1\output26\model-best")
answer = []
for doc in docs:
    docx = nlp_(doc)
    #print(docx.ents)
    res = []
    for ent in docx.ents:
        res.append((ent.label_, ent.text))
    answer.append(res)
jsonStr = json.dumps(answer) 

print(jsonStr)
#return jsonStr

[[["NORDER", "2340838"], ["CONTRACT", "SR-1-06"], ["CONTRACT1", "1327"], ["AMOUNT", "*"], ["POS", "1000"], ["AMOUNT", "100"], ["ARTICLE", "6010001836"], ["PRICE", "54,52"], ["UNIT", "100"], ["SUMMA", "54,52"], ["TARIFF", "73079910"], ["COUNTRY", "Germany\n\n"], ["PR_SURCH", "6,90"], ["SURCHARGE", "3,76"], ["TOTAL", "58,28"]], [["NORDER", "2340912"], ["CONTRACT", "SR-1-06"], ["CONTRACT1", "1328"], ["POS", "1100"], ["AMOUNT", "100"], ["ARTICLE", "6010001836"], ["PRICE", "54,52"], ["UNIT", "100"], ["SUMMA", "54,52"], ["TARIFF", "73079910"], ["COUNTRY", "Germany\n\n"], ["PR_SURCH", "6,90"], ["SURCHARGE", "3,76"], ["TOTAL", "58,28"]], [["NORDER", "2341067"], ["CONTRACT", "SR-1-06"], ["CONTRACT1", "1329"], ["POS", "1200"], ["AMOUNT", "100"], ["ARTICLE", "6010001836"], ["PRICE", "54,52"], ["UNIT", "100"], ["SUMMA", "54,52"], ["TARIFF", "73079910"], ["COUNTRY", "Germany\n\n"], ["PR_SURCH", "6,90"], ["SURCHARGE", "3,76"], ["TOTAL", "58,28"]], [["NORDER", "2341290"], ["CONTRACT", "SR-1-06"], ["C

In [25]:
def pdfDocToString(f_name):
    size = 2400
    pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"
    f_pdf = open(f_name, 'rb').read()
    images = convert_from_bytes(f_pdf, dpi = 300, size=size)
    images = convert_from_path(f_name, dpi = 300)
    text = ''
    for img in images:
        text += pytesseract.image_to_string(img , lang='rus+eng', config='--psm 4' )
    return text

In [5]:
def shortening(text):
    nlp = spacy.blank("en")

    doc = nlp(text)

    matcher = Matcher(nlp.vocab)

    pattern3 = [
        {'LOWER':'CONTRIBUTING'.lower()},
        {'LOWER':'TO'.lower()}, 
        {'LOWER':'YOUR'.lower()}, 
        {'LOWER':'SUCCESS'.lower()}
        ]
    matcher.add('ENDPAGE', [pattern3])

    pattern4 = [
        {'LOWER':'article'},
        {'LOWER':'code'}, 
        {'LOWER':'price'}, 
        {'LOWER':'eur'},
        {'LOWER':'unit'},
        {'LOWER':'amount'},
        {'LOWER':'eur'}
    ]

    pattern4_ = [{'LOWER':'russische'}, {'LOWER':'foed'}]
    matcher.add('STARTPAGE', [pattern4, pattern4_])

    pattern5  = [
        {'LOWER':'doc.no'},
        {'LOWER':'.'},
        {'LOWER':'/'},
        {'LOWER':'date'},
        {'LOWER':'page'}
    ]
    
    pattern6  = [
        {'LOWER':'doc.по'},
        {'LOWER':'.'},
        {'LOWER':'/'},
        {'LOWER':'date'},
        {'LOWER':'page'}
    ] 
    matcher.add('ABOUTNOPAGE', [pattern5, pattern6])

    doc = nlp(text)

    matches = matcher(doc)
    spans = [Span(doc, start, end, label=match_id) for match_id, start, end in matches]

    paging = []

    pageinfo = {'ABOUTNOPAGE':0, 'STARTPAGE':0, 'ENDPAGE':0}
    shortenText = ''
    firstPage = True
    txt = '''import spacy
    from spacy.tokens import Span

    nlp = spacy.blank("en")\n'''
    docs  = []
    npos = 0
    npage = 0
    for sp in spans:
        if sp.label_  == 'ABOUTNOPAGE':
            pageinfo[sp.label_] = sp.end
        elif sp.label_  == 'STARTPAGE':
            pageinfo[sp.label_] = sp.end
        elif sp.label_  == 'ENDPAGE':
            pageinfo[sp.label_] = sp.start
            paging.append(pageinfo)
            
            if firstPage ==True:
                shortenText = doc[:pageinfo['ENDPAGE']].text
                firstPage = False
                docs.append(doc[:pageinfo['ENDPAGE']].text)
                txt = txt + strSeparator + doc[:pageinfo['ENDPAGE']].text 
            else:
                shortenText = shortenText +  doc[pageinfo['STARTPAGE']:pageinfo['ENDPAGE']].text
                docs.append(doc[pageinfo['STARTPAGE']:pageinfo['ENDPAGE']].text)
                print(pageinfo, pageinfo['ENDPAGE'] - pageinfo['STARTPAGE'])
                txt = txt + strSeparator + doc[pageinfo['STARTPAGE']:pageinfo['ENDPAGE']].text 

                
                #break
            pageinfo = {'ABOUTNOPAGE':0, 'STARTPAGE':0, 'ENDPAGE':0}
            npage += 1

        #print(pageinfo)
    #print(txt)
    print(len(paging))
    return shortenText

#with open(fName.replace('.txt', 's.txt'), encoding="utf8", mode='w') as f:
#    f.write(txt)


In [6]:
catName = fName.split('\\')[-1].split('.')[0]
print('\\'.join(fName.split('\\')[:-1]))
if os.path.isdir(catName):
    print('Dir is')

F:\Python\OCR_diploma\docs_base\StauffInvoices


In [ ]:

text = pdfDocToString(f_name=fName)
txtS = shortening(text)

print(txtS)

In [8]:
def text2spans(rtest):
    rtest = rtest.replace(strSeparator, '')

    nlp = spacy.blank("en")
    matcher = Matcher(nlp.vocab)

    patternPurchase = [{'LOWER': 'purchase'}, 
        {'LOWER':'order'}, 
        {'LOWER':'number'}, 
        {'LOWER': ':'}]
    matcher.add('PTRPURCHASE', [patternPurchase])

    patternTariff = [{'LOWER': 'customs'}, 
        {'LOWER':'tariff'}, 
        {'IS_ALPHA': True},
        {'IS_PUNCT': True, 'OP': '*'}, 
        {'IS_DIGIT': True}]
    matcher.add('PTRTARIFF', [patternTariff])

    patternCountry = [{'LOWER': 'country'}, 
        {'LOWER':'of'} , 
        {'LOWER': 'origin'}, 
        {'LOWER': ':'}, 
        {'IS_ALPHA': True}]
    matcher.add('PTRCOUNTRY', [patternCountry])

    patternPC = [{'LIKE_NUM': True}, {'LIKE_NUM': True},
        {'LOWER' : {'in' : ['pc', 'рс']}},
        {'LIKE_NUM':True}    ] 

    matcher.add('PTROFPC', [patternPC])

    doc1 = nlp(rtest)

    #print("Verbs:", [token.text for token in doc1[45:54]])

    matches = matcher(doc1)
    spans = [Span(doc1, start, end, label=match_id) for match_id, start, end in matches]

    startSpan = 0
    endSpan = 0
    #posTariff = 0
    #txt2 = 'import json \nimport spacy \nfrom spacy.matcher import Matcher \nfrom spacy.tokens import Span, DocBin \ndocs = [] \n'
    #nPos = 0
    docs = []
    pos2PC = 0
    for sp in spans:
        if sp.label_  == 'PTRPURCHASE':
            startSpan = sp.start
            
        #elif sp.label_  == 'PTRTARIFF':             posTariff = sp.end-1
            
        #elif sp.label_  == 'PTROFPC':             pos2PC = sp.start
        
        elif sp.label_  == 'PTRCOUNTRY':
            endSpan = sp.end
            #pos2PC = pos2PC - startSpan  
            

            docs.append(doc1[startSpan:endSpan].text)
            
    return docs

In [27]:
docs = text2spans(txtS)

In [12]:
type(docs[1])

str

In [28]:
nlp_ = spacy.load(r"C:\Users\AdminAsus\source\python\spacyST\spacySt-1\output\model-best")
answer = []
for doc in docs:
    docx = nlp_(doc)
    #print(docx.ents)
    res = []
    for ent in docx.ents:
        res.append((ent.label_, ent.text))
    answer.append(res)
jsonStr = json.dumps(answer)    
print(len(answer))
print(jsonStr)
    #for ent in docx.ents:
    #    print(ent.text, ent.label_)

49
[[["CONTRACT", "SR-1-06"], ["CONTRACT1", "1327"], ["POS", "1000"], ["AMOUNT", "100"], ["ARTICLE", "6010001836"], ["PRICE", "54,52"], ["UNIT", "100"], ["SUM", "54,52"], ["TARIFF", "73079910"], ["COUNTRY", "Germany"]], [["CONTRACT", "SR-1-06"], ["CONTRACT1", "1328"], ["POS", "1100"], ["AMOUNT", "100"], ["ARTICLE", "6010001836"], ["PRICE", "54,52"], ["UNIT", "100"], ["SUM", "54,52"], ["TARIFF", "73079910"], ["COUNTRY", "Germany"]], [["CONTRACT", "SR-1-06"], ["CONTRACT1", "1329"], ["POS", "1200"], ["AMOUNT", "100"], ["ARTICLE", "6010001836"], ["PRICE", "54,52"], ["UNIT", "100"], ["SUM", "54,52"], ["TARIFF", "73079910"], ["COUNTRY", "Germany"]], [["CONTRACT", "SR-1-06"], ["CONTRACT1", "1330"], ["POS", "1300"], ["AMOUNT", "100"], ["ARTICLE", "6010001836"], ["PRICE", "54,52"], ["UNIT", "100"], ["SUM", "54,52"], ["TARIFF", "73079910"], ["COUNTRY", "Germany"]], [["CONTRACT", "SR-1-06"], ["CONTRACT1", "1331"], ["POS", "1400"], ["AMOUNT", "100"], ["ARTICLE", "6010001836"], ["PRICE", "54,52"], 

In [30]:
import json
def transfer(fName):
    text = pdfDocToString(f_name=fName)
    txtS = shortening(text)
    docs = text2spans(txtS)
    nlp_ = spacy.load(r"C:\Users\AdminAsus\source\python\spacyST\spacySt-1\output\model-best")
    answer = []
    for doc in docs:
        docx = nlp_(doc)
        #print(docx.ents)
        res = []
        for ent in docx.ents:
            res.append((ent.label_, ent.text))
        answer.append(res)
    jsonStr = json.dumps(answer) 
    return jsonStr

In [31]:
print(transfer(fName=fName))

{'ABOUTNOPAGE': 585, 'STARTPAGE': 620, 'ENDPAGE': 878} 258
{'ABOUTNOPAGE': 1106, 'STARTPAGE': 1143, 'ENDPAGE': 1401} 258
{'ABOUTNOPAGE': 1629, 'STARTPAGE': 1664, 'ENDPAGE': 1922} 258
{'ABOUTNOPAGE': 2150, 'STARTPAGE': 2185, 'ENDPAGE': 2440} 255
{'ABOUTNOPAGE': 2668, 'STARTPAGE': 2705, 'ENDPAGE': 2934} 229
{'ABOUTNOPAGE': 3162, 'STARTPAGE': 3197, 'ENDPAGE': 3429} 232
{'ABOUTNOPAGE': 3657, 'STARTPAGE': 3694, 'ENDPAGE': 3933} 239
{'ABOUTNOPAGE': 4161, 'STARTPAGE': 4196, 'ENDPAGE': 4428} 232
{'ABOUTNOPAGE': 4656, 'STARTPAGE': 4693, 'ENDPAGE': 4911} 218
{'ABOUTNOPAGE': 5139, 'STARTPAGE': 5174, 'ENDPAGE': 5409} 235
{'ABOUTNOPAGE': 5637, 'STARTPAGE': 5674, 'ENDPAGE': 5901} 227
{'ABOUTNOPAGE': 6129, 'STARTPAGE': 6166, 'ENDPAGE': 6398} 232
{'ABOUTNOPAGE': 6626, 'STARTPAGE': 6663, 'ENDPAGE': 6880} 217
{'ABOUTNOPAGE': 7108, 'STARTPAGE': 7145, 'ENDPAGE': 7391} 246
{'ABOUTNOPAGE': 7619, 'STARTPAGE': 7656, 'ENDPAGE': 7901} 245
{'ABOUTNOPAGE': 8129, 'STARTPAGE': 8166, 'ENDPAGE': 8362} 196
{'ABOUTNOPA